In [ ]:
! pip install datasets

In [ ]:
! pip install transformers

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np

# get data from huggingdface

In [3]:
from datasets import load_dataset
data = load_dataset("code_search_net","javascript")

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 123889
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 6483
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 8253
    })
})

In [4]:
dataset_train = data['train'].remove_columns(["repository_name", "func_path_in_repository","func_name", "whole_func_string","language", "func_code_tokens","func_documentation_tokens", "split_name","func_code_url"])
dataset_test = data['test'].remove_columns(["repository_name", "func_path_in_repository","func_name", "whole_func_string","language", "func_code_tokens","func_documentation_tokens", "split_name","func_code_url"])


In [16]:
dataset_train

Dataset({
    features: ['func_code_string', 'func_documentation_string'],
    num_rows: 123889
})

In [5]:
train_data = pd.DataFrame(dataset_train)
test_data =  pd.DataFrame(dataset_test)

train_data= train_data.rename(columns={"func_code_string":'output',"func_documentation_string":'input'})
test_data= test_data.rename(columns={"func_code_string":'output',"func_documentation_string":'input'})

In [25]:
train_data.head(5)

,output,input
0,function createTypeScriptLanguageService(optio...,"#region Discovery, LanguageService & Setup"
1,function discoverAndReadFiles(options) {\n ...,Read imports and follow them until all files h...
2,"function getRealNodeSymbol(checker, node) {\n ...",#endregion #region Utils \nReturns the node's ...
3,"function shouldSkipAlias(node, declaration) {\...",Go to the original declaration for cases: (1)...
4,"function getTokenAtPosition(sourceFile, positi...",Get the token whose text contains the position


In [6]:
train_data = train_data[['input', 'output']]
test_data = test_data[['input', 'output']]

In [27]:
train_data.head(5)

,input,output
0,"#region Discovery, LanguageService & Setup",function createTypeScriptLanguageService(optio...
1,Read imports and follow them until all files h...,function discoverAndReadFiles(options) {\n ...
2,#endregion #region Utils \nReturns the node's ...,"function getRealNodeSymbol(checker, node) {\n ..."
3,Go to the original declaration for cases: (1)...,"function shouldSkipAlias(node, declaration) {\..."
4,Get the token whose text contains the position,"function getTokenAtPosition(sourceFile, positi..."


In [7]:
del(data,dataset_train,dataset_test)

# train

In [8]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [9]:
batch_size=8
num_of_batches=int(len(train_data)/batch_size)

In [10]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-base")
# tokenizer = AutoTokenizer.from_pretrained("JulesBelveze/t5-small-headline-generator")
# model = AutoModelForSeq2SeqLM.from_pretrained("JulesBelveze/t5-small-headline-generator")

In [12]:
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [13]:
from transformers import Adafactor
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [14]:
def trainfun(model,train_df,batch_size,num_of_epochs,optimizer):
  model.train()
  num_of_batches=int(len(train_df)/batch_size)
  loss_per_10_steps=[]
  for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
    running_loss=0
    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
      # print(f'Batch {i}')
      inputbatch=[]
      labelbatch=[]
      new_df=train_df[i*batch_size:i*batch_size+batch_size]
      for indx,row in new_df.iterrows():
        input = str(row['input']) + '[SEP]' 
        labels = row['output']+'[SEP]'   
        inputbatch.append(input)
        labelbatch.append(labels)
      inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=350,return_tensors='pt',truncation =True)["input_ids"]
      labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=350,return_tensors="pt",truncation =True) ["input_ids"]
      inputbatch=inputbatch.to(dev)
      labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
      optimizer.zero_grad()

    # Forward propogation
      outputs = model(input_ids=inputbatch, labels=labelbatch)
      loss = outputs.loss
      loss_num=loss.item()
      logits = outputs.logits
      running_loss+=loss_num
      if i%10 ==0:      
        loss_per_10_steps.append(loss_num)
      out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
      loss.backward()

    #updating the params
      optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  return model,loss_per_10_steps

In [15]:
model,loss_per_10_steps=trainfun(model,train_data,batch_size,15,optimizer)

Running epoch: 1


OutOfMemoryError: ignored

In [ ]:
torch.save(model.state_dict(), "t5_model.pt")

In [ ]:
def prediction(model, sent):
  model.eval()
  sent1 = str(sent) + "[SEP]"
  input_ids = tokenizer.encode(sent1, return_tensors="pt",max_length=512,padding=True)
  input_ids=input_ids.to(dev)
  outputs = model.generate(input_ids,num_beams=8, do_sample=True, min_length=10, max_length=512)
  #print(outputs)
  z=tokenizer.decode(outputs[0],skip_special_tokens=True,min_length=512)
  return(z)

In [ ]:
test_data['predicted']=test_data['input'].apply(lambda x: prediction(model,x))
test_data['predicted']=test_data['predicted'].apply(lambda x: x.replace('[SEP]',''))
test_data.to_csv('res_testset.csv')

In [ ]:
test_data[["input","predicted"]]

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
# tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")


# def paraphrase(s):   
#     context = s
#     text = "paraphrase: "+context + " </s>"

#     encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
#     input_ids,attention_mask  = encoding["input_ids"].to(dev), encoding["attention_mask"].to(dev)
#     model.eval()
#     diverse_beam_outputs = model.generate(
#         input_ids=input_ids,attention_mask=attention_mask,
#         max_length=128,
#         early_stopping=True,
#         num_beams=5,
#         num_beam_groups = 5,
#         num_return_sequences=1,
#         diversity_penalty = 0.70
#     )
# #     print ("Original: ",context)
#     for beam_output in diverse_beam_outputs:
#         sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
#         return (sent)
    
# test_data['paraphrased'] = test_data['predicted'].apply(paraphrase)

# deployed

In [ ]:
model_name = "text2imagev3"
description = f"""
THE FIRST TEST
"""

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, ModelCard, create_repo, get_full_repo_name

# Set up repo and upload files
hub_model_id = get_full_repo_name(model_name)
create_repo(hub_model_id)
api = HfApi()
api.upload_folder(folder_path=args.output_dir, path_in_repo="", repo_id=hub_model_id)

content = f"""
---
license: creativeml-openrail-m
tags:
- pytorch
- diffusers
- stable-diffusion
- text-to-image
- diffusion-models-class
- dreambooth-hackathon


## Usage

```python
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained('{hub_model_id}')
image = pipeline().images[0]
image
```
"""

card = ModelCard(content)
hub_url = card.push_to_hub(hub_model_id)